## Implementation of Infinite Latent Feature Models and the Indian Buffet Process

**Indian Buffet Process (IBP)**

The Indian Buffet is an adaptation of Chinese Buffett Process where each object instead of being associated with a single latent class can  be associated with multiple classes. This is particularly useful when each object has multile latent features and by associating objects with a single class we cannot partition them into homogeneous subsets.

In the Indian buffet process, $N$ customers enter a restaurant one after another. Each customer encounters a buffet 
consisting of infinitely many dishes arranged in a line. The first customer starts at the left of the buffet and 
takes a serving from each dish, stopping after a Poisson($\alpha$) number of dishes. The $i$th customer moves along the buffet, 
sampling dishes in proportion to their popularity, taking dish $k$ with probability $\frac{m_k}{i}$ , where $m_k$ is the number of 
previous customers who have sampled that dish. Having reached the end of all previous sampled dishes, the $i$th customer 
then tries a Poisson($\frac{\alpha}{i}$) number of new dishes. Which costumer chose which dishes is indicated using a binary matrix **Z** with $N$ rows and infinitely many columns(corresponding to the infinitely many selection of dished), where $z_{ik}$ = 1 if the $i$th costumer sampled $k$th dish.

IBP can be used as a prior in models for unsupervised learning. An example of which is presentd in the paper by Griffiths and Ghahramani, where IBP is used as a prior in linear-Gaussian binary latent feature model.

### Algorithm

* Gamma prior for $\alpha$
$$
\alpha \sim Gamma(1,1)
$$
* Prior on **Z** is obtained by IBP as:
$$
P(z_{ik}=1|\textbf{z}_{-i,k}) = \frac{n_{-i,k}}{N}
$$

* Likelihood is given by
\begin{equation}
P(X|Z,\sigma_X, \sigma_A) = \frac{1}{(2 \pi)^{ND/2} (\sigma_X)^{(N-K)D}(\sigma_A)^{KD}(|Z^TZ+\frac{\sigma_X^2}{\sigma_A^2}I|)^{D/2}}exp\{-\frac{1}{2\sigma_X^2}tr(X^T(I-Z(Z^TZ+\frac{\sigma_X^2}{\sigma_A^2}I)^{-1}Z^T)X)\}
\end{equation}

After we have the likelihood and the prior given by IBP,

* full conditional posterior for **Z** can be calculated as:
$$
P(z_{ik}|X,Z_{-(i,k)},\sigma_X,\sigma_A) \propto  P(X|Z,\sigma_X, \sigma_A) * P(z_{ik}=1|\textbf{z}_{-i,k}) 
$$

To sample the number of new features for observation $i$, we use a truncated distribution, computing probabilities for a range of values $K_1^{(i)}$ up to an upper bound (say 4). The prior on number of features is given by $Poisson(\frac{\alpha}{N})$.
Using this prior and the likelihood, we sample the nummber of new features.

* Full conditional posterior for $\alpha$ is given by:
$$
P(\alpha|Z) \sim Gamma(1+K_+,1+\sum_{i=1}^{N} H_i)
$$

* For $\sigma_X$ and $\sigma_A$, we use MH algorithm as follows:
\begin{eqnarray}
\epsilon \sim Uniform(-.05,.05)\\
\sigma_X^{*} =  \sigma_X +\epsilon\\
\end{eqnarray}
Accept this new $\sigma_X$ with probability given by:
$$
AR = min\{1,\frac{Likelihood(X|\sigma_X^{*},...)}{Likelihood(X|\sigma_X,...)}\}\\
$$
Where AR is the acceptance ratio. We use similar algorithm to sample $\sigma_A$

#### Pseudo-code for a single iteration of Gibbs and MH combined algorigthm

In [79]:
import numpy as np
import scipy as sp
import math
import matplotlib.pyplot as plt
from __future__ import division
plt.style.use('ggplot')
import Image
import matplotlib.cm as cm
%matplotlib inline
%precision 4
import time

###### Simulate data

In [80]:
np.random.seed(1)

N = 100 #number of objects
K = 4 #true number of features
D = 36 # dimension of feature


sigmaX0 = .5;
A = np.array((0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, \
             0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, \
             0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, \
             0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0)).reshape(4, D)


I = (sigmaX0)*np.identity(D)
Z0 = np.zeros((N, K))
X = np.zeros((N, D))
for i in range(N):
    Z0[i,:] = (np.random.uniform(0,1,K) > .5).astype(int)
    while (np.sum(Z0[i,:]) == 0):
        Z0[i,:] = (np.random.uniform(0,1,K) > .5).astype(int)
    #X[i,:] = np.random.multivariate_normal(Z0[i,:].dot(A), I)
    #X(i,:) = randn(1, object_dim)*I+Z_orig(i,:)*A;
    X[i,:] = np.random.normal(0,1, (1,D)).dot(I)+Z0[i,:].dot(A)
# plt.figure(num=None, figsize=(12,3), dpi=80, facecolor='w', edgecolor='k')
# plt.subplot(141)
# plt.pcolormesh(A[0,:].reshape(6,6),cmap=plt.cm.gray)     
# plt.subplot(142)
# plt.pcolormesh(A[1,:].reshape(6,6),cmap=plt.cm.gray)  
# plt.subplot(143)
# plt.pcolormesh(A[2,:].reshape(6,6),cmap=plt.cm.gray)  
# plt.subplot(144)
# plt.pcolormesh(A[3,:].reshape(6,6),cmap=plt.cm.gray) 

###### Set initial numbers, dimensions and features

###### Sample prior

In [81]:
np.random.seed(1)
def sampleIBP(alpha, N):
    result = np.zeros((N, 1000))
    t = np.random.poisson(alpha)
    if t>0:
        result[0,0:t] = np.ones(t)
    Kplus = t
    for i in range(1,N):
        for j in range(Kplus):
            p = np.sum(result[0:i,j])/(i+1)
            if np.random.uniform(0,1) < p:
                result[i,j] = 1
        t = np.random.poisson(alpha/(i+1))
        if t>0:
            result[i,Kplus:Kplus+t] = np.ones(t)
            Kplus = Kplus+t
    result = result[:,0:Kplus]
    return np.array((result, Kplus))


In [73]:
def calcInverse(Z,M,i,k,val):
    M_i = M - M.dot(Z[i,:].T.dot(Z[i,:].dot(M)))/(Z[i,:].dot(M.dot(Z[i,:].T))-1)
    #M_i = M-np.dot(np.dot(np.dot(M,Zn[i,:].T),Zn[i,:]),M)/(np.dot(np.dot(Zn[i,:],M),Zn[i,:].T)-1)
    #Zn[i,k] = val
    Z[i,k] = val
    M = M_i - M_i.dot(Z[i,:].T.dot(Z[i,:].dot(M_i)))/(Z[i,:].dot(M_i.dot(Z[i,:].T))+1)
    #M = M_i-np.dot(np.dot(np.dot(M_i,Zn[i,:].T),Zn[i,:]),M_i)/(np.dot(np.dot(Zn[i,:],M_i),Zn[i,:].T)+1)
    Inv = M
    return Inv

##### Likelihood function

In [4]:
# define a log likelihood function 
def ll(X, Z, sigmaX, sigmaA, K, D, N):
    #M = Z[:,0:K].T.dot(Z[:,0:K])+sigmaX**2/sigmaA**2*np.identity(K)
    M = Z.T.dot(Z)+(sigmaX**2/sigmaA**2)*np.identity(K)
    return (-1)*np.log(2*np.pi)*N*D*.5 - np.log(sigmaX)*(N-K)*D - np.log(sigmaA)*K*D - .5*D*np.log(np.linalg.det(M)) \
        -.5/(sigmaX**2)*np.trace( (X.T.dot( np.identity(N)-Z.dot(np.linalg.inv(M).dot(Z.T)) )).dot(X) )

In [74]:
# define a log likelihood function 
def ll_m(X, Z, sigmaX, sigmaA, K, D, N, M):
    M1 = Z.T.dot(Z)+(sigmaX**2/sigmaA**2)*np.identity(K)
    return (-1)*np.log(2*np.pi)*N*D*.5 - np.log(sigmaX)*(N-K)*D - np.log(sigmaA)*K*D - .5*D*np.log(np.linalg.det(M1)) \
        -.5/(sigmaX**2)*np.trace( (X.T.dot( np.identity(N)-Z.dot(M.dot(Z.T)) )).dot(X) )

In [ ]:
%timeit ll(X, Z, sigmaX, sigmaA, Kplus , D, N)
%timeit cy_ll(X, Z, sigmaX, sigmaA, Kplus , D, N)

In [75]:
HN = 0.
for i in range(1,N+1):
    HN += 1./i
    
#Kplus = 4 #current number of features with at least one object
niter = 400
sigmaX = 1.
sigmaA = 1.
alpha = 1.
maxNew = 4
BURN_IN=200

In [76]:

SAMPLE_SIZE=niter-BURN_IN

K_inf=20

chain_Z=np.zeros((SAMPLE_SIZE,N,K_inf))
chain_K=np.zeros((SAMPLE_SIZE,1))
chain_sigma_X=np.zeros((SAMPLE_SIZE,1))
chain_sigma_A=np.zeros((SAMPLE_SIZE,1))
chain_alpha=np.zeros((SAMPLE_SIZE,1))

In [78]:

t0 = time.time()
np.random.seed(1)
Z, Kplus = sampleIBP(alpha, N)
s_counter=0

for j in range(niter):
    print("iteration:",j ,  "Kplus:",Kplus,  "shape of Z", Z.shape, "alpha:", alpha, "sigmaX", sigmaX)
    #update z
    if((j+1)>BURN_IN):
        chain_Z[s_counter,:,0:Kplus]=Z
        chain_K[s_counter]=Kplus
        chain_sigma_X[s_counter]=sigmaX
        chain_sigma_A[s_counter]=sigmaA
        chain_alpha[s_counter]=alpha
        s_counter=s_counter+1
    
    for i in range(N):
        M = np.linalg.inv(Z.T.dot(Z)+(sigmaX**2/sigmaA**2)*np.identity(Kplus))
        for k in range(Kplus):
            #print k
            if k>=Kplus:
                break     
            #Removing the singular features, i.e. the ones that have 1 for the current object only.
            if Z[i,k] > 0:
                if (np.sum(Z[:,k])- 1) <=0:
                    #Z[i,k] = 0
                    Z[:,k:(Kplus-1)] = Z[:,(k+1):Kplus] #shift everything one column to the left
                    Kplus = Kplus-1
                    Z = Z[:,0:Kplus] # remove the last column as it is redundent
                    M = np.linalg.inv(Z.T.dot(Z)+(sigmaX**2/sigmaA**2)*np.identity(Kplus))
                    continue #We're no longer looking at this feature, so move to another one               
        
            P = np.zeros(2)
            
            M0 = calcInverse(Z,M,i,k,0)
            M1 = calcInverse(Z,M,i,k,1)
            #set Z[i,k] = 0 and calculate posterior probability
            Z[i,k] = 0
            P[0] = ll_m(X, Z, sigmaX, sigmaA, Kplus, D, N, M0) + np.log(N-np.sum(Z[:,k])) - np.log(N)
        
            #set Z[i,k] = 1 and calculate posterior probability
            Z[i,k] = 1
            P[1] = ll_m(X, Z,sigmaX, sigmaA, Kplus, D, N, M1)  + np.log(np.sum(Z[:,k])- 1) - np.log(N)
        
            P = np.exp(P - max(P))
            U = np.random.uniform(0,1)
            if U<(P[1]/(np.sum(P))):
                Z[i,k] = 1
                M = M1
            else:
                Z[i,k] = 0  
                M = M0
    
    
        #Sample number of new features
        prob = np.zeros(maxNew)
        alphaN = alpha/N
        for kNew in range(maxNew): # max new features is 3
            Z_temp = Z
            if kNew>0:
                addCols = np.zeros((N,kNew))
                addCols[i,:] = 1
                Z_temp = np.hstack((Z_temp, addCols))
            
            pois = kNew*np.log(alphaN) - alphaN - np.log(math.factorial(kNew))
            M = np.linalg.inv(Z_temp.T.dot(Z_temp)+(sigmaX**2/sigmaA**2)*np.identity(Kplus+kNew))
            lik = ll_m(X = X, Z = Z_temp, sigmaX = sigmaX, sigmaA = sigmaA, K=(Kplus+kNew), D= D, N= N, M=M)
            prob[kNew] = pois + lik

        #normalize prob
        prob = np.exp(prob - max(prob))
        prob = prob/sum(prob)
        
        U = np.random.uniform(0,1,1)
        p = 0
        kNew=0
        for new in range(maxNew):
            p = p+prob[new]
            if U<p:
                kNew = new
                break
     
        #Add kNew new columns to Z and set the values at ith row to 1 for all of them
        if kNew>0:
            addCols = np.zeros((N,kNew))
            addCols[i,:] = 1
            Z = np.hstack((Z, addCols))
        Kplus = Kplus + kNew 
    
    M = np.linalg.inv(Z.T.dot(Z)+(sigmaX**2/sigmaA**2)*np.identity(Kplus))
    
    llCurrent = ll_m(X, Z, sigmaX, sigmaA, Kplus, D, N, M )
    #update sigmaX
    if np.random.uniform(0,1) < .5:
        sigmaX_new = sigmaX - np.random.uniform(0,1)/20
    else:
        sigmaX_new = sigmaX + np.random.uniform(0,1)/20
    llNew = ll_m(X, Z, sigmaX_new, sigmaA, Kplus, D, N, M)

    arX = np.exp(min(0,llNew-llCurrent))
    U = np.random.uniform(0,1)
    if U < arX:
        sigmaX = sigmaX_new
        
        
    #update sigma_A
    #epsA = np.random.uniform(0,1)
    if np.random.uniform(0,1) < .5:
        sigmaA_new = sigmaA - np.random.uniform(0,1)/20
    else:
        sigmaA_new = sigmaA + np.random.uniform(0,1)/20
    
    
#     epsA = np.random.uniform(-.05,.05)
#     sigmaA_new = sigmaA+epsA
    #llCurrent = ll(X, Z, sigmaX, sigmaA, Kplus, D, N )
    #llCurrent = ll(X, Z, sigmaX, sigmaA, Kplus, D, N )
    llNew = ll_m(X, Z, sigmaX, sigmaA_new, Kplus, D, N, M)

    arA = np.exp(min(0,llNew-llCurrent))

    U = np.random.uniform(0,1)
    if U < arA:
        sigmaA = sigmaA_new
        
    alpha = np.random.gamma(1+Kplus, 1/(1+HN))
t1 = time.time()
timeElapsed = t1-t0

('iteration:', 0, 'Kplus:', 0, 'shape of Z', (100, 0), 'alpha:', 0.1146895879722918, 'sigmaX', 0.7432472715385191)


/home/bitnami/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:4: RuntimeWarning: invalid value encountered in log


LinAlgError: Arrays cannot be empty

In [19]:
def sampler(X, niter, BURN_IN, sigmaX, sigmaA,alpha, N, D, maxNew):
    HN = 0.
    for i in range(1,N+1):
        HN += 1./i

    SAMPLE_SIZE=niter-BURN_IN

    K_inf=20

    chain_Z=np.zeros((SAMPLE_SIZE,N,K_inf))
    chain_K=np.zeros((SAMPLE_SIZE,1))
    chain_sigma_X=np.zeros((SAMPLE_SIZE,1))
    chain_sigma_A=np.zeros((SAMPLE_SIZE,1))
    chain_alpha=np.zeros((SAMPLE_SIZE,1))
    np.random.seed(1)
    Z, Kplus = sampleIBP(alpha, N)
    s_counter=0

    for j in range(niter):
        if j%50==0:
            print j
        #print("iteration:",j ,  "Kplus:",Kplus,  "shape of Z", Z.shape, "alpha:", alpha, "sigmaX", sigmaX)
        #update z
        if((j+1)>BURN_IN):
            chain_Z[s_counter,:,0:Kplus]=Z
            chain_K[s_counter]=Kplus
            chain_sigma_X[s_counter]=sigmaX
            chain_sigma_A[s_counter]=sigmaA
            chain_alpha[s_counter]=alpha
            s_counter=s_counter+1

        for i in range(N):
            for k in range(Kplus):
                #print k
                if k>=Kplus:
                    break     
                #Removing the singular features, i.e. the ones that have 1 for the current object only.
                if Z[i,k] > 0:
                    if (np.sum(Z[:,k])- 1) <=0:
                        #Z[i,k] = 0
                        Z[:,k:(Kplus-1)] = Z[:,(k+1):Kplus] #shift everything one column to the left
                        Kplus = Kplus-1
                        Z = Z[:,0:Kplus] # remove the last column as it is redundent
                        continue #We're no longer looking at this feature, so move to another one               

                P = np.zeros(2)
                #set Z[i,k] = 0 and calculate posterior probability
                Z[i,k] = 0
                P[0] = ll(X, Z, sigmaX, sigmaA, Kplus, D, N) + np.log(N-np.sum(Z[:,k])) - np.log(N)

                #set Z[i,k] = 1 and calculate posterior probability
                Z[i,k] = 1
                P[1] = ll(X, Z,sigmaX, sigmaA, Kplus, D, N)  + np.log(np.sum(Z[:,k])- 1) - np.log(N)

                P = np.exp(P - max(P))
                U = np.random.uniform(0,1)
                if U<(P[1]/(np.sum(P))):
                    Z[i,k] = 1
                else:
                    Z[i,k] = 0   


            #Sample number of new features
            prob = np.zeros(maxNew)
            alphaN = alpha/N
            for kNew in range(maxNew): # max new features is 3
                Z_temp = Z
                if kNew>0:
                    addCols = np.zeros((N,kNew))
                    addCols[i,:] = 1
                    Z_temp = np.hstack((Z_temp, addCols))

                pois = kNew*np.log(alphaN) - alphaN - np.log(math.factorial(kNew))
                lik = ll(X = X, Z = Z_temp, sigmaX = sigmaX, sigmaA = sigmaA, K=(Kplus+kNew), D= D, N= N)
                prob[kNew] = pois + lik

            #normalize prob
            prob = np.exp(prob - max(prob))
            prob = prob/sum(prob)

            U = np.random.uniform(0,1,1)
            p = 0
            kNew=0
            for new in range(maxNew):
                p = p+prob[new]
                if U<p:
                    kNew = new
                    break

            #Add kNew new columns to Z and set the values at ith row to 1 for all of them
            if kNew>0:
                addCols = np.zeros((N,kNew))
                addCols[i,:] = 1
                Z = np.hstack((Z, addCols))
            Kplus = Kplus + kNew 

        llCurrent = ll(X, Z, sigmaX, sigmaA, Kplus, D, N )
        #update sigmaX
        if np.random.uniform(0,1) < .5:
            sigmaX_new = sigmaX - np.random.uniform(0,1)/20
        else:
            sigmaX_new = sigmaX + np.random.uniform(0,1)/20
        llNew = ll(X, Z, sigmaX_new, sigmaA, Kplus, D, N)

        arX = np.exp(min(0,llNew-llCurrent))
        U = np.random.uniform(0,1)
        if U < arX:
            sigmaX = sigmaX_new


        #update sigma_A
        #epsA = np.random.uniform(0,1)
        if np.random.uniform(0,1) < .5:
            sigmaA_new = sigmaA - np.random.uniform(0,1)/20
        else:
            sigmaA_new = sigmaA + np.random.uniform(0,1)/20


    #     epsA = np.random.uniform(-.05,.05)
    #     sigmaA_new = sigmaA+epsA
        #llCurrent = ll(X, Z, sigmaX, sigmaA, Kplus, D, N )
        #llCurrent = ll(X, Z, sigmaX, sigmaA, Kplus, D, N )
        llNew = ll(X, Z, sigmaX, sigmaA_new, Kplus, D, N)

        arA = np.exp(min(0,llNew-llCurrent))

        U = np.random.uniform(0,1)
        if U < arA:
            sigmaA = sigmaA_new

        alpha = np.random.gamma(1+Kplus, 1/(1+HN))
    
    return(chain_K)

In [8]:
HN = 0.
for i in range(1,N+1):
    HN += 1./i
    
#Kplus = 4 #current number of features with at least one object
niter = 400
sigmaX = 1.
sigmaA = 1.
alpha = 1.
maxNew = 4
BURN_IN=200
t0 = time.time()
#sampler(X, niter, BURN_IN, sigmaX, sigmaA,alpha, N, D, maxNew)
t1= time.time()

dt = t1-t0
dt

0.0002

In [9]:
%load_ext Cython

In [23]:
%%file Cython_setup.py
from distutils.core import setup
from Cython.Build import cythonize

setup(
    ext_modules = cythonize("Cython_functions.pyx")
)

Overwriting Cython_setup.py


In [28]:
%%file Cython_functions.pyx
import numpy as np

def ll(X, Z, sigmaX, sigmaA, K, D, N):
    #M = Z[:,0:K].T.dot(Z[:,0:K])+sigmaX**2/sigmaA**2*np.identity(K)
    M = Z.T.dot(Z)+(sigmaX**2/sigmaA**2)*np.identity(K)
    return (-1)*np.log(2*np.pi)*N*D*.5 - np.log(sigmaX)*(N-K)*D - np.log(sigmaA)*K*D - .5*D*np.log(np.linalg.det(M)) \
        -.5/(sigmaX**2)*np.trace( (X.T.dot( np.identity(N)-Z.dot(np.linalg.inv(M).dot(Z.T)) )).dot(X) )

np.random.seed(1)
def sampleIBP(alpha, N):
    result = np.zeros((N, 1000))
    t = np.random.poisson(alpha)
    if t>0:
        result[0,0:t] = np.ones(t)
    Kplus = t
    for i in range(1,N):
        for j in range(Kplus):
            p = np.sum(result[0:i,j])/(i+1)
            if np.random.uniform(0,1) < p:
                result[i,j] = 1
        t = np.random.poisson(alpha/(i+1))
        if t>0:
            result[i,Kplus:Kplus+t] = np.ones(t)
            Kplus = Kplus+t
    result = result[:,0:Kplus]
    return np.array((result, Kplus))

Overwriting Cython_functions.pyx


In [29]:
! python Cython_setup.py build_ext --inplace

Compiling Cython_functions.pyx because it changed.
Cythonizing Cython_functions.pyx
running build_ext
building 'Cython_functions' extension
gcc -pthread -fno-strict-aliasing -g -O2 -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/home/bitnami/anaconda/include/python2.7 -c Cython_functions.c -o build/temp.linux-x86_64-2.7/Cython_functions.o
gcc -pthread -shared build/temp.linux-x86_64-2.7/Cython_functions.o -L/home/bitnami/anaconda/lib -lpython2.7 -o /home/bitnami/STA663-FinalProject/Cython_functions.so


In [31]:
%%cython -a
import numpy as np
cimport numpy as np
import scipy as sp
import math
import Cython_functions as func

def sampler_cy(X, niter, BURN_IN, sigmaX, sigmaA,alpha, N, D, maxNew):
    HN = 0.
    for i in range(1,N+1):
        HN += 1./i

    SAMPLE_SIZE=niter-BURN_IN

    K_inf=20

    chain_Z=np.zeros((SAMPLE_SIZE,N,K_inf))
    chain_K=np.zeros((SAMPLE_SIZE,1))
    chain_sigma_X=np.zeros((SAMPLE_SIZE,1))
    chain_sigma_A=np.zeros((SAMPLE_SIZE,1))
    chain_alpha=np.zeros((SAMPLE_SIZE,1))
    np.random.seed(1)
    Z, Kplus = func.sampleIBP(alpha, N)
    s_counter=0

    for j in range(niter):
        if j%50==0:
            print j
        #print("iteration:",j ,  "Kplus:",Kplus,  "shape of Z", Z.shape, "alpha:", alpha, "sigmaX", sigmaX)
        #update z
        if((j+1)>BURN_IN):
            chain_Z[s_counter,:,0:Kplus]=Z
            chain_K[s_counter]=Kplus
            chain_sigma_X[s_counter]=sigmaX
            chain_sigma_A[s_counter]=sigmaA
            chain_alpha[s_counter]=alpha
            s_counter=s_counter+1

        for i in range(N):
            for k in range(Kplus):
                #print k
                if k>=Kplus:
                    break     
                #Removing the singular features, i.e. the ones that have 1 for the current object only.
                if Z[i,k] > 0:
                    if (np.sum(Z[:,k])- 1) <=0:
                        #Z[i,k] = 0
                        Z[:,k:(Kplus-1)] = Z[:,(k+1):Kplus] #shift everything one column to the left
                        Kplus = Kplus-1
                        Z = Z[:,0:Kplus] # remove the last column as it is redundent
                        continue #We're no longer looking at this feature, so move to another one               

                P = np.zeros(2)
                #set Z[i,k] = 0 and calculate posterior probability
                Z[i,k] = 0
                P[0] = func.ll(X, Z, sigmaX, sigmaA, Kplus, D, N) + np.log(N-np.sum(Z[:,k])) - np.log(N)

                #set Z[i,k] = 1 and calculate posterior probability
                Z[i,k] = 1
                P[1] = func.ll(X, Z,sigmaX, sigmaA, Kplus, D, N)  + np.log(np.sum(Z[:,k])- 1) - np.log(N)

                P = np.exp(P - max(P))
                U = np.random.uniform(0,1)
                if U<(P[1]/(np.sum(P))):
                    Z[i,k] = 1
                else:
                    Z[i,k] = 0   


            #Sample number of new features
            prob = np.zeros(maxNew)
            alphaN = alpha/N
            for kNew in range(maxNew): # max new features is 3
                Z_temp = Z
                if kNew>0:
                    addCols = np.zeros((N,kNew))
                    addCols[i,:] = 1
                    Z_temp = np.hstack((Z_temp, addCols))

                pois = kNew*np.log(alphaN) - alphaN - np.log(math.factorial(kNew))
                lik = func.ll(X = X, Z = Z_temp, sigmaX = sigmaX, sigmaA = sigmaA, K=(Kplus+kNew), D= D, N= N)
                prob[kNew] = pois + lik

            #normalize prob
            prob = np.exp(prob - max(prob))
            prob = prob/sum(prob)

            U = np.random.uniform(0,1,1)
            p = 0
            kNew=0
            for new in range(maxNew):
                p = p+prob[new]
                if U<p:
                    kNew = new
                    break

            #Add kNew new columns to Z and set the values at ith row to 1 for all of them
            if kNew>0:
                addCols = np.zeros((N,kNew))
                addCols[i,:] = 1
                Z = np.hstack((Z, addCols))
            Kplus = Kplus + kNew 

        llCurrent = func.ll(X, Z, sigmaX, sigmaA, Kplus, D, N )
        #update sigmaX
        if np.random.uniform(0,1) < .5:
            sigmaX_new = sigmaX - np.random.uniform(0,1)/20
        else:
            sigmaX_new = sigmaX + np.random.uniform(0,1)/20
        llNew = func.ll(X, Z, sigmaX_new, sigmaA, Kplus, D, N)

        arX = np.exp(min(0,llNew-llCurrent))
        U = np.random.uniform(0,1)
        if U < arX:
            sigmaX = sigmaX_new


        #update sigma_A
        #epsA = np.random.uniform(0,1)
        if np.random.uniform(0,1) < .5:
            sigmaA_new = sigmaA - np.random.uniform(0,1)/20
        else:
            sigmaA_new = sigmaA + np.random.uniform(0,1)/20


    #     epsA = np.random.uniform(-.05,.05)
    #     sigmaA_new = sigmaA+epsA
        #llCurrent = ll(X, Z, sigmaX, sigmaA, Kplus, D, N )
        #llCurrent = ll(X, Z, sigmaX, sigmaA, Kplus, D, N )
        llNew = func.ll(X, Z, sigmaX, sigmaA_new, Kplus, D, N)

        arA = np.exp(min(0,llNew-llCurrent))

        U = np.random.uniform(0,1)
        if U < arA:
            sigmaA = sigmaA_new

        alpha = np.random.gamma(1+Kplus, 1/(1+HN))
    
    return(chain_K)

### Profiling

In [58]:
HN = 0.
for i in range(1,N+1):
    HN += 1./i
    
#Kplus = 4 #current number of features with at least one object
niter = 400
sigmaX = 1.
sigmaA = 1.
alpha = 1.
maxNew = 4
BURN_IN=200
t0 = time.time()
#sampler(X, niter, BURN_IN, sigmaX, sigmaA,alpha, N, D, maxNew)
# t1= time.time()

# dt = t1-t0
# dt

In [33]:
%timeit sampler_cy(X, 300, 0, sigmaX, sigmaA,alpha, N, D, maxNew)
%timeit sampler(X, 300, 0, sigmaX, sigmaA,alpha, N, D, maxNew)

0
50
100
150
200
250
0
50
100
150
200
250
0
50
100
150
200
250
0
50
100
150
200
250
1 loops, best of 3: 1min 37s per loop
0
50
100


KeyboardInterrupt: 

In [57]:
stats = %prun -r -q sampler(X, 30, 0, sigmaX, sigmaA,alpha, N, D, maxNew)

0


/home/bitnami/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:5: RuntimeWarning: invalid value encountered in log


LinAlgError: Arrays cannot be empty

In [35]:
stats.sort_stats('time').print_stats(10);

         19735386 function calls in 110.447 seconds

   Ordered by: internal time
   List reduced from 120 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  1500360   41.488    0.000   41.488    0.000 {method 'dot' of 'numpy.ndarray' objects}
   300072   14.871    0.000   96.889    0.000 <ipython-input-4-b6095f8d1143>:2(ll)
   300072    9.032    0.000   14.822    0.000 linalg.py:455(inv)
        1    6.954    6.954  110.447  110.447 <ipython-input-19-2fbc1bc057f9>:1(sampler)
   300072    6.007    0.000   11.067    0.000 linalg.py:1679(det)
   600144    3.941    0.000    9.605    0.000 numeric.py:2125(identity)
   600144    3.535    0.000    5.664    0.000 twodim_base.py:190(eye)
   300072    3.293    0.000    3.293    0.000 {method 'trace' of 'numpy.ndarray' objects}
   809740    2.623    0.000    2.623    0.000 {numpy.core.multiarray.zeros}
   600144    1.776    0.000    3.090    0.000 linalg.py:139(_commonType)




In [59]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [61]:
lstats = %lprun -r -f sampler sampler(X, 30, 0, sigmaX, sigmaA,alpha, N, D, maxNew)

0


In [ ]:
sigmaXZ=chain_Z[SAMPLE_SIZE-1,:,0:10].reshape(100,10)
sigma_X=chain_sigma_X[SAMPLE_SIZE-1]
sigma_A=chain_sigma_A[SAMPLE_SIZE-1]
A_inf=np.dot(np.dot(np.linalg.inv((np.dot(Z.T,Z)+(sigma_X/sigma_A)*np.eye(4))),Z.T),X)

In [ ]:
plt.figure(num=None, figsize=(12,3), dpi=80, facecolor='w', edgecolor='k')
plt.subplot(141)
plt.pcolormesh(A_inf[0,:].reshape(6,6),cmap=plt.cm.gray)     
plt.subplot(142)
plt.pcolormesh(A_inf[1,:].reshape(6,6),cmap=plt.cm.gray)  
plt.subplot(143)
plt.pcolormesh(A_inf[2,:].reshape(6,6),cmap=plt.cm.gray)  
plt.subplot(144)
plt.pcolormesh(A_inf[3,:].reshape(6,6),cmap=plt.cm.gray)

In [ ]:
#plt.plot(chain_K)
#np.mean(chain_K)
#np.sum(chain_K[200:999]==6)
plt.hist(chain_K, bins = [4,5,6,7,8,9,10])

In [ ]:
plt.plot(chain_alpha)
np.mean(chain_alpha)

In [ ]:
plt.plot(chain_sigma_X)
np.mean(chain_sigma_X)

In [ ]:
plt.plot(chain_sigma_A)
np.mean(chain_sigma_A)

In [ ]:
from numbapro import cuda, vectorize, guvectorize, check_cuda
from numbapro import void, uint8 , uint32, uint64, int32, int64, float32, float64, f8

### Unit testing

To check if the code is working properly some of the unit testings I've come up with so far are:
* Probabilities calculated for the presence of feature have to be between 0 and 1.
* 